In [2]:
pip install /root/CodeCommit/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/CodeCommit/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
msspackages is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from msspackages import setup_runner
setup_runner()

b"Hit:1 http://deb.debian.org/debian buster InRelease\nHit:2 http://security.debian.org/debian-security buster/updates InRelease\nHit:3 http://deb.debian.org/debian buster-updates InRelease\nReading package lists...\nBuilding dependency tree...\nReading state information...\n59 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nsudo is already the newest version (1.8.27-1+deb10u4).\n0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.\nReading package lists...\nBuilding dependency tree...\nReading state information...\ndefault-jre is already the newest version (2:1.11-71).\n0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.\nReading package lists...\nBuilding dependency tree...\nReading state information...\npython3 is already the newest version (3.7.3-1).\n0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.\nReading package lists...\nBuilding depend

In [3]:
from msspackages import Pyspark_data_ingestion

In [2]:
from feature_engineering import node_autoencoder_ad_preprocessing, node_autoencoder_ad_feature_engineering, node_autoencoder_train_test_split
from feature_engineering import container_autoencoder_ad_preprocessing, container_autoencoder_ad_feature_engineering, container_autoencoder_train_test_split
from feature_engineering import pod_autoencoder_ad_preprocessing, pod_autoencoder_ad_feature_engineering, pod_autoencoder_train_test_split
import pandas as pd

In [4]:
data_ingestion = Pyspark_data_ingestion()
spark = data_ingestion.get_spark()


In [6]:
pod_test_df = spark.read.parquet('s3a://emr-serverless-output-pd/aa/test_data1/')

In [11]:
#pod_test_df.count()

In [3]:
def pod_training_data_builder():
    """
    Output
    ------
        pod_training_data:df
        training df to be stored in s3

        pod_testing_data:df
        testing df to be stores in s3

        pod_training_tensor: np array
        tensor stored in s3 for model training

        pod_testing_tensor: np array
        tensor stored in s3 for model testing

    """
    
    #pre processing
    pod_features_data, pod_processed_data = pod_autoencoder_ad_preprocessing("pod_autoencoder_ad","11-30-2022","2022","10","10","10")
    
    #test, train split
    pod_train_data, pod_test_data = pod_autoencoder_train_test_split(pod_processed_data)

    #Train data feature engineering
    pod_training_data = pod_autoencoder_ad_feature_engineering(pod_features_data, pod_train_data)
 
    #Test data feature engineering
    pod_testing_data = pod_autoencoder_ad_feature_engineering(pod_features_data, pod_test_data)

    return pod_training_data, pod_training_tensor, pod_testing_data, pod_testing_tensor


In [4]:
%%time
pod_training_data, pod_training_tensor, pod_testing_data, pod_testing_tensor = pod_training_data_builder()


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.7/site-packages/py4j/clientserver.py", line 540, in send_command
    "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPyth

ConnectionRefusedError: [Errno 111] Connection refused

In [23]:
def training_data_builder(preprocessing_args,
                          preprocessing_fnc, 
                          preprocessing_train_test_split_fnc,
                          feature_engineering_fnc):
    
    #pre processing
    features_data, processed_data = preprocessing_fnc(*preprocessing_args)
    #pd.set_option('display.max_columns', None)  
    print(features_data.head())
    print(processed_data.show(truncate=False))
    
    #test, train split
    train_data, test_data = preprocessing_train_test_split_fnc(processed_data)

    #Train data feature engineering
    training_data = feature_engineering_fnc(features_data, train_data)
    #s3_operations.write_to_s3(node_training_data, s3_path)
    print(training_data.show(truncate=False))
 
    #Test data feature engineering
    testing_data = feature_engineering_fnc(features_data, test_data)
    #s3_operations.write_to_s3(node_testing_data, s3_path)
    print(testing_data.show(truncate=False))
    
    return training_data, testing_data

def pod_training_data_builder():
    
    #pre processing
    pod_features_data, pod_processed_data = pod_autoencoder_ad_preprocessing("pod_autoencoder_ad","11-30-2022","2022","10","10","10")
    #pd.set_option('display.max_columns', None)  
    print(pod_features_data.head())
    print(pod_processed_data.show(truncate=False))
    
    #test, train split
    pod_train_data, pod_test_data = pod_autoencoder_train_test_split(pod_processed_data)

    #Train data feature engineering
    pod_training_data = pod_autoencoder_ad_feature_engineering(pod_features_data, pod_train_data)
    #s3_operations.write_to_s3(node_training_data, s3_path)
    print(pod_training_data.show(truncate=False))
 
    #Test data feature engineering
    pod_testing_data = pod_autoencoder_ad_feature_engineering(pod_features_data, pod_test_data)
    #s3_operations.write_to_s3(node_testing_data, s3_path)
    print(pod_testing_data.show(truncate=False))
    
    return pod_training_data, pod_testing_data

In [ ]:
#build training data for pod
pod_training_data, pod_testing_data =training_data_builder(preprocessing_args = ["pod_autoencoder_ad","11-30-2022","2022","10","10","10"],
                                                           preprocessing_fnc = autoencoder_ad_preprocessing, 
                                                           preprocessing_train_test_split_fnc = pod_autoencoder_train_test_split,
                                                           feature_engineering_fnc = pod_autoencoder_ad_feature_engineering)